In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.functional import F
import random
from treeqn_traj_simplest_2 import TreeQN
from torch.optim import Adam
from image_world_2 import get_train_test_data
import pandas as pd

In [2]:
a = torch.zeros(5,5)
b = torch.zeros(5,5)
b[0][0] = 1
baseline_loss = F.mse_loss(a,b).item()
print(baseline_loss)


0.03999999910593033


<h1>Checkerboard

In [3]:
stacked_train_trajectories, stacked_test_trajectories = get_train_test_data(distance=4) #distance agent starts from goal

<h1>Model

In [4]:
input_shape = stacked_train_trajectories[0].shape# minimum size #train_data[0][0].shape
num_actions = 4
tree_depth = 4
embedding_dim = 64
gamma = 1 
decode_dropout = 0.5
t1 =True#True is Einsum. False +dx 
model = TreeQN(input_shape=input_shape, num_actions=num_actions, tree_depth=tree_depth, embedding_dim=embedding_dim, gamma=gamma,decode_dropout=decode_dropout,t1=t1)
cuz_encoder = model.encoder
cuz_decoder = model.decoder

Einsum Transiton


In [5]:
# z = cuz_encoder(stacked_train_trajectories[0])
# print(z.shape)
# z = cuz_decoder(z)
# print(z.shape)

<h1>Autoencoder Training

In [6]:
# optimizer = Adam(list(cuz_encoder.parameters())+list(cuz_decoder.parameters()), lr=1e-4,weight_decay=1e-5)
# max_val_loss = 1000
# max_val_iter=-1
# for i in range(100): 
#     cuz_decoder.train()
#     avg_loss =0
#     train_count = 0
#     valid_count = 0
#     for traj in random.sample(stacked_train_trajectories, len(stacked_train_trajectories)):
#         noisy_traj = traj.clone() + torch.randn_like(traj) * 0.1
#         train_count +=1
#         encodeded_tensor = cuz_encoder(noisy_traj)
#         decoded_tensor = cuz_decoder(encodeded_tensor)#model.decoder(encodeded_tensor)
#         loss = F.mse_loss(decoded_tensor, noisy_traj)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         avg_loss += loss.item()
#     if i % 10 ==0:
#         cuz_decoder.eval()
#         valid_loss = 0
#         with torch.no_grad():
#             for traj in stacked_test_trajectories:
#                 valid_count +=1
#                 encodeded_tensor = cuz_encoder(traj)
#                 decoded_tensor = cuz_decoder(encodeded_tensor)#model.decoder(encodeded_tensor)
#                 loss = F.mse_loss(decoded_tensor, traj)
#                 valid_loss += loss.item()
#         if (valid_loss/valid_count) < max_val_loss:
#             max_val_loss = valid_loss/valid_count
#             max_val_iter = i
#         print('VALID LOSS',i,(valid_loss/valid_count)/baseline_loss)
#         print('Train Loss',i,(avg_loss/train_count)/baseline_loss,'\n') #Want less than 1

In [7]:
# losses = []
# for traj in stacked_test_trajectories:
#     for tensor in traj:
#         cuz_decoder.eval()
#         tensor = tensor.unsqueeze(0)
#         encodeded_tensor = cuz_encoder(tensor)
#         decoded_tensor = cuz_decoder(encodeded_tensor) 
#         loss = F.mse_loss(torch.round(decoded_tensor), tensor)
#         losses.append(loss.item())
# print(pd.Series(losses).value_counts())
# test_train = stacked_test_trajectories[0][0].unsqueeze(0)
# encodeded_tensor = cuz_encoder(test_train)
# decoded_tensor = cuz_decoder(encodeded_tensor)
# print(torch.round(test_train[0]))
# print(torch.round(decoded_tensor[0]))

<h1>Train IRL

In [8]:
all_gradients = []
optimizer = Adam(model.parameters(), lr=1e-4)
weighted=False
for epoch in range(3000):  # epochs
    avg_loss = 0
    avg_decode_loss, avg_first_loss, avg_second_loss, avg_third_loss, avg_fourth_loss = 0, 0, 0, 0, 0
    model.train()
    for traj in random.sample(stacked_train_trajectories, len(stacked_train_trajectories)):  # sample through all data in random order each epoch
        # Get reconstruction loss to help ground abstract state
        ###NO NOISE YET!
        decoded_values, transition_probabilities = model(traj[0].unsqueeze(0))
        decode_loss = F.mse_loss(decoded_values[0], traj[0].unsqueeze(0))

        # Flatten transition probabilities to then weigh with loss of each predicted state at each layer
        first = transition_probabilities[0].view(-1,1,1,1)
        second = transition_probabilities[1].view(-1,1,1,1)
        third = transition_probabilities[2].view(-1,1,1,1)
        fourth = transition_probabilities[3].view(-1,1,1,1)

        if weighted:
            #Weighted Transitions
            first_loss = (F.mse_loss(decoded_values[1], traj[1].unsqueeze(0), reduction='none') * first).sum()
            second_loss = (F.mse_loss(decoded_values[2], traj[2].unsqueeze(0), reduction='none') * second).sum()
            third_loss = (F.mse_loss(decoded_values[3], traj[3].unsqueeze(0), reduction='none') * third).sum()
            fourth_loss = (F.mse_loss(decoded_values[4], traj[4].unsqueeze(0), reduction='none') * fourth).sum()
        else:
            #Greedy Policy (Squeezing to eliminate batch and channel dimensions)
            first_loss = (F.mse_loss(decoded_values[1][first.argmax()].squeeze(0),traj[1].squeeze(0)))
            second_loss = (F.mse_loss(decoded_values[2][second.argmax()].squeeze(0),traj[2].squeeze(0)))
            third_loss = (F.mse_loss(decoded_values[3][third.argmax()].squeeze(0),traj[3].squeeze(0)))
            fourth_loss = (F.mse_loss(decoded_values[4][fourth.argmax()].squeeze(0),traj[4].squeeze(0)))

        total_loss = first_loss + second_loss  + third_loss  + fourth_loss + decode_loss

        # break if total loss is nan
        if torch.isnan(total_loss):
            raise ValueError("NAN LOSS")


        avg_decode_loss += decode_loss.item()
        avg_first_loss += first_loss.item()
        avg_second_loss += second_loss.item()
        avg_third_loss += third_loss.item()
        avg_fourth_loss += fourth_loss.item()
        avg_loss += total_loss.item()


        optimizer.zero_grad()
        total_loss.backward()
        # Monitor gradients before clipping and stepping
        #all_gradients.append(image_world.store_gradients(model))
        optimizer.step()

    if epoch % 10 == 0: 
        model.eval()
        avg_val_loss = 0
        for traj in stacked_test_trajectories:
            with torch.no_grad():
                decoded_values, transition_probabilities = model(traj[0].unsqueeze(0))
                decode_loss = F.mse_loss(decoded_values[0], traj[0].unsqueeze(0))

                # Flatten transition probabilities to then weigh with loss of each predicted state at each layer
                first = transition_probabilities[0].view(-1,1,1,1)
                second = transition_probabilities[1].view(-1,1,1,1)
                third = transition_probabilities[2].view(-1,1,1,1)
                fourth = transition_probabilities[3].view(-1,1,1,1)

                if weighted:
                    #Weighted Transitions
                    first_loss = (F.mse_loss(decoded_values[1], traj[1].unsqueeze(0), reduction='none') * first).sum()
                    second_loss = (F.mse_loss(decoded_values[2], traj[2].unsqueeze(0), reduction='none') * second).sum()
                    third_loss = (F.mse_loss(decoded_values[3], traj[3].unsqueeze(0), reduction='none') * third).sum()
                    fourth_loss = (F.mse_loss(decoded_values[4], traj[4].unsqueeze(0), reduction='none') * fourth).sum()
                else:
                    #Greedy Policy (Squeezing to eliminate batch and channel dimensions)
                    first_loss = (F.mse_loss(decoded_values[1][first.argmax()].squeeze(0),traj[1].squeeze(0)))
                    second_loss = (F.mse_loss(decoded_values[2][second.argmax()].squeeze(0),traj[2].squeeze(0)))
                    third_loss = (F.mse_loss(decoded_values[3][third.argmax()].squeeze(0),traj[3].squeeze(0)))
                    fourth_loss = (F.mse_loss(decoded_values[4][fourth.argmax()].squeeze(0),traj[4].squeeze(0)))

                avg_val_loss += (first_loss + second_loss  + third_loss  + fourth_loss + decode_loss).item()
        print("Val Loss", (avg_val_loss/(len(stacked_test_trajectories)*5))/baseline_loss)

        #print just validation

    #Individual Lossses
    avg_decode_loss = (avg_decode_loss / len(stacked_train_trajectories))/baseline_loss
    avg_first_loss = (avg_first_loss / len(stacked_train_trajectories))/baseline_loss
    avg_second_loss = (avg_second_loss / len(stacked_train_trajectories))/baseline_loss
    avg_third_loss = (avg_third_loss / len(stacked_train_trajectories))/baseline_loss
    avg_fourth_loss = (avg_fourth_loss / len(stacked_train_trajectories))/baseline_loss
    #Full Loss
    avg_total_loss = (avg_loss / (len(stacked_train_trajectories)*5))/baseline_loss

    print(f"Epoch {epoch + 1}, Total Loss: {avg_total_loss}, DLoss: {avg_decode_loss}, A1: {avg_first_loss}, A2: {avg_second_loss}, A3: {avg_third_loss}, A4: {avg_fourth_loss}")


Val Loss 2.622971264503768
Epoch 1, Total Loss: 3.746865301824388, DLoss: 4.3535253031823995, A1: 3.968190602963609, A2: 3.60785776574821, A3: 3.6984902594474187, A4: 3.106262592108342
Epoch 2, Total Loss: 2.374169383824219, DLoss: 2.327236233071188, A1: 2.476067557181094, A2: 2.576101445861136, A3: 2.583318334144133, A4: 1.9081233531619557
Epoch 3, Total Loss: 1.9501329271009245, DLoss: 1.821340847850587, A1: 2.110959266285146, A2: 2.293524402951651, A3: 2.163997848100339, A4: 1.360842300047582
Epoch 4, Total Loss: 1.7974400933546308, DLoss: 1.6134775611503909, A1: 1.966388443593308, A2: 2.1402133448011145, A3: 2.066536860236719, A4: 1.2005842593199278
Epoch 5, Total Loss: 1.6887659452637067, DLoss: 1.4791449250003765, A1: 1.8317938271136003, A2: 2.0220130926190145, A3: 1.9775171346522111, A4: 1.1333607349335968
